In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
import numpy as np

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('data/for_PCA_dong_ver2_rec.csv')
df['수거함_결핍률'] = 1 - (df['수거함_개수']/df['총_거주인구수'])
df.head()
# df.columns

,자치구,행정동,행정동_코드,월평균_총생활인구수,연령대0_평균,연령대10_평균,연령대20_평균,연령대30_평균,연령대40_평균,연령대50_평균,연령대60_평균,연령대70이상_평균,1인가구,2인가구,3인가구,4인가구,5인가구,6인가구,7인 이상 가구,연령대_100,연령대_0_거주인구수,연령대_10_거주인구수,연령대_20_거주인구수,연령대_30_거주인구수,연령대_40_거주인구수,연령대_50_거주인구수,연령대_60_거주인구수,연령대_70_거주인구수,연령대_80_거주인구수,연령대_90_거주인구수,유흥주점영업,단란주점,제과점영업,집단급식소,일반음식점,휴게음식점,연령대_2030_거주인구수_합,연령대_2030_거주인구수_평균,연령대_20_거주인구수_비율,연령대_30_거주인구수_비율,연령대_2030_거주인구수_비율,1인가구_비율,식품접객업,주점,주요경제활동인구_거주인구수,주요경제활동인구_생활인구수,PET예측량,수거함_개수,총_거주인구수,수거함개수_당_거주인구수,수거함_결핍률
0,광진구,중곡4동,11215770,1.878735e+07,1.235867e+06,1.636002e+06,2.489974e+06,2.873870e+06,2.659556e+06,3.082496e+06,2.444396e+06,2.365192e+06,3747,3366,2560,1760,390,83,24,12.916667,1248.250000,1717.750000,4008.416667,3921.750000,3693.666667,4808.916667,4136.500000,2087.916667,732.000000,92.166667,0.0,12.0,81.0,142.0,1763.0,341.0,7930.166667,7930.166667,0.151488,0.148213,0.299701,0.314082,2339.0,12.0,16432.750000,1.110590e+07,95.289109,0.0,26460.250000,26460.250000,1.000000
1,광진구,구의3동,11215870,2.288022e+07,1.850343e+06,2.443886e+06,3.166729e+06,3.539066e+06,3.946832e+06,3.543896e+06,2.372386e+06,2.017077e+06,3110,2871,2559,2093,394,63,21,5.333333,1657.083333,2269.666667,4193.833333,4211.833333,4071.166667,4424.916667,3294.000000,1558.666667,690.583333,100.666667,24.0,48.0,136.0,137.0,3152.0,1020.0,8405.666667,8405.666667,0.158391,0.159071,0.317462,0.279903,4517.0,72.0,16901.750000,1.419652e+07,99.004343,1.0,26477.750000,26477.750000,0.999962
2,동대문구,용신동,11230536,2.853400e+07,1.133191e+06,1.487169e+06,4.944984e+06,4.914170e+06,4.421953e+06,4.547594e+06,3.490275e+06,3.594664e+06,8464,3551,2035,1273,279,71,29,8.666667,1123.083333,1279.833333,5677.250000,5000.000000,3798.833333,4239.000000,3894.916667,2305.333333,1051.500000,121.500000,84.0,264.0,72.0,336.0,7255.0,1476.0,10677.250000,10677.250000,0.199202,0.175439,0.374641,0.539040,9487.0,348.0,18715.083333,1.882870e+07,190.213604,0.0,28499.916667,28499.916667,1.000000
3,동대문구,전농1동,11230560,2.357436e+07,1.457401e+06,1.907768e+06,4.067471e+06,3.419103e+06,3.547157e+06,3.468014e+06,2.742669e+06,2.964774e+06,5814,3212,2419,1667,370,79,23,13.500000,1670.583333,1693.333333,4182.916667,3837.500000,3893.583333,4445.000000,3915.166667,2274.166667,1006.416667,114.750000,60.0,84.0,167.0,315.0,5179.0,1938.0,8020.416667,8020.416667,0.154654,0.141883,0.296537,0.428004,7743.0,144.0,16359.000000,1.450175e+07,136.932032,0.0,27046.916667,27046.916667,1.000000
4,동대문구,답십리1동,11230600,1.676137e+07,1.560452e+06,1.286225e+06,2.085826e+06,2.953395e+06,2.825986e+06,2.435188e+06,1.816438e+06,1.797864e+06,3717,2869,2577,1828,470,82,24,12.333333,2465.750000,1812.916667,3795.333333,4793.000000,4354.416667,3892.583333,3082.333333,1625.916667,738.916667,90.500000,96.0,101.0,117.0,231.0,3436.0,1157.0,8588.333333,8588.333333,0.142339,0.179755,0.322095,0.321345,5138.0,197.0,16835.333333,1.030039e+07,91.659854,1.0,26664.000000,26664.000000,0.999962


### 1. VIF로 다중공선성 체크
- 10이 넘는 변수 제거
- 초기 컬럼 선택 : '월평균_총생활인구수', '1인가구',
       '2인가구', '3인가구', '4인가구', '5인가구','유흥주점영업', '단란주점', '제과점영업', '집단급식소',
       '일반음식점', '휴게음식점', '연령대_2030_거주인구수_합', '연령대_2030_거주인구수_평균',
       '연령대_20_거주인구수_비율', '연령대_30_거주인구수_비율', '연령대_2030_거주인구수_비율', '1인가구_비율',
       '식품접객업', '주점', '주요경제활동인구_거주인구수', '주요경제활동인구_생활인구수', 'PET예측량', '수거함_개수',
       '총_거주인구수', '거주인구_당_수거함개수'

- 다중공선성으로 필터링 결과 : ['1인가구_비율','식품접객업', '주점', '주요경제활동인구_생활인구수', '수거함_개수','거주인구_당_수거함개수']
- 참고 : 상수항 (intercept)의 다중공선성이 높을 경우, 일반적으로 큰 문제가 되지는 않습니다. 

In [3]:
x_features = ['1인가구_비율','식품접객업','연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수']

# x_features에 대한 VIF 계산을 위해 상수(intercept) 열을 추가
x_features_with_const = sm.add_constant(df[x_features])

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Variable"] = x_features_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(x_features_with_const.values, i) for i in range(x_features_with_const.shape[1])]
vif_data

,Variable,VIF
0,const,23.584170
1,1인가구_비율,2.028202
2,식품접객업,2.305992
3,연령대_2030_거주인구수_합,2.016161
4,주요경제활동인구_생활인구수,2.152316


### 2. Best Subset Selection
- 모든 가능한 독립 변수의 조합을 고려하여 최적의 모델을 선택하는 방법 
- 우리는 AIC 를 기반으로 
- AIC (Akaike Information Criterion):
AIC는 모델의 적합도와 모델의 복잡성을 고려하여 모델 선택을 합니다. <br>
AIC가 낮을수록 모델의 예측 성능이 좋으며, 데이터에 더 잘 적합됩니다.<br>
데이터에 대한 적합성을 높이는 데 초점을 맞추고, 모델의 복잡성을 상대적으로 덜 고려할 때 사용됩니다.<br>
**작은 크기의 데이터셋 또는 더 유연한 모델 선택**을 할 때 유용합니다.


- BIC (Bayesian Information Criterion):
BIC는 AIC와 마찬가지로 적합도와 복잡성을 고려하지만, 모델의 크기에 민감하게 반응합니다.<br>
BIC가 낮을수록 모델의 예측 성능이 좋으며, 데이터에 더 잘 적합됩니다. 그러나 BIC는 모델의 크기를 더 강력하게 제한합니다.<br>
데이터에 대한 적합성을 고려하면서, 불필요한 변수를 제한하고 모델의 단순성을 강조할 때 사용됩니다.<br>
**큰 크기의 데이터셋 또는 변수 선택에 제한**을 두고 싶을 때 유용합니다.

In [4]:
import itertools
import statsmodels.api as sm

# 종속 변수 설정
y = df['PET예측량']

# 독립 변수 리스트 (const 를 제외한 변수들)
independent_variables = vif_data.iloc[1:, 0].tolist()

# 최적 모델 초기화
best_model = None
best_model_summary = None
best_aic = float('inf')

best_bic_model = None
best_bic_model_summary = None
best_bic = float('inf')

# Best Subset Selection
for k in range(1, len(independent_variables) + 1):
    for subset in itertools.combinations(independent_variables, k):
        X_subset = df[list(subset)]
        X_subset = sm.add_constant(X_subset)
        model = sm.OLS(y, X_subset).fit()
        
        aic = model.aic
        bic = model.bic
        
        if aic < best_aic:
            best_aic = aic
            best_aic_model = model
            best_aic_model_summary = model.summary()
        
        if bic < best_bic:
            best_bic = bic
            best_bic_model = model
            best_bic_model_summary = model.summary()

print("### Best Subset Selection 결과 (AIC):")
print(best_aic_model_summary)

# print('\n\n')

# print("### Best Subset Selection 결과 (BIC):")
# print(best_bic_model_summary)

### Best Subset Selection 결과 (AIC):
                            OLS Regression Results                            
Dep. Variable:                 PET예측량   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     1276.
Date:                Sat, 28 Oct 2023   Prob (F-statistic):           4.62e-68
Time:                        00:31:25   Log-Likelihood:                -273.21
No. Observations:                  85   AIC:                             554.4
Df Residuals:                      81   BIC:                             564.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
cons

### 최종변수 + 추가변수 >> minmaxScaling >> PCA
- 추가변수 : (1 - 수거함수 / 거주인구수) = 수거함 결핍률
- 주성분 분석은 변수 간의 선형 결합으로 주성분을 구성

In [5]:
# df['수거함_결핍률'] = 1 - (df['수거함_개수']/df['총_거주인구수'])

In [24]:
# pca 수행
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

for_pca = ['1인가구_비율', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수']
pre_pca = df[for_pca]

# 스케일링
scaler = MinMaxScaler()
pre_pca = scaler.fit_transform(pre_pca)
pre_pca = pd.DataFrame(pre_pca)

# PCA 모델 생성
pca = PCA()

# 데이터를 PCA 모델에 fitting
pca.fit(pre_pca)

# 개별 주성분의 설명 분산 비율 출력
explained_variance_ratio = pca.explained_variance_ratio_
print("## 개별 주성분의 설명 분산 비율:\n", explained_variance_ratio)

# 누적 분산 비율 출력
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
print("## 누적 분산 비율:\n", cumulative_variance_ratio, '\n')

# 주성분(PC) 확인
principal_components = pca.components_

# 주성분의 Loadings 확인
loadings = pca.explained_variance_ratio_

# 주성분과 Loadings 출력
for i, (pc, loading) in enumerate(zip(principal_components, loadings), 1):
    print(f'주성분 {i}: {pc} (Explained Variance: {loading:.2f})')

## 개별 주성분의 설명 분산 비율:
 [0.45077724 0.41387217 0.13535058]
## 누적 분산 비율:
 [0.45077724 0.86464942 1.        ] 

주성분 1: [0.18192496 0.69455688 0.69605607] (Explained Variance: 0.45)
주성분 2: [ 0.88015491  0.20060505 -0.430215  ] (Explained Variance: 0.41)
주성분 3: [-0.43844115  0.69090401 -0.57482259] (Explained Variance: 0.14)


### 주성분 1 loading (소수점 3째자리에서 반올림, 절댓값)
- '1인가구_비율' : 0.34805601 >> 0.65
- '연령대_2030_거주인구수_합' : 0.72730218 >> 0.60
- '주요경제활동인구_생활인구수' : 0.54630551 >> 0.46
- '수거함_결핍률' : -0.22680131 >> 0.23 (폐기)

### 행정동 별 지표 산출
- 전 변수 minmaxScaling
- 지표 계산

In [22]:
# minMax스케일링
fin_features = ['자치구', '행정동', '행정동_코드', '1인가구_비율', '연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수']
df = df[fin_features]

scaler = MinMaxScaler()
df.iloc[:, 3:] = scaler.fit_transform(df.iloc[:, 3:])
df['평가점수'] = df['1인가구_비율']*0.46 + df['연령대_2030_거주인구수_합']*0.60 + df['주요경제활동인구_생활인구수']*0.50 +  df['수거함개수_당_거주인구수']*0.42 
df['평가점수'] = np.round(df['평가점수'], 2)
# df.head()
df.sort_values(by='평가점수', ascending = False).head()

,자치구,행정동,행정동_코드,1인가구_비율,연령대_2030_거주인구수_합,주요경제활동인구_생활인구수,수거함개수_당_거주인구수,평가점수
41,강서구,화곡1동,11500540,0.581286,1.000000,0.573449,0.849013,1.51
60,관악구,청룡동,11620595,0.988142,0.878827,0.468999,0.245140,1.32
51,영등포구,영등포동,11560535,1.000000,0.600803,0.745670,0.010561,1.20
28,은평구,진관동,11380690,0.258060,0.526623,0.670115,1.000000,1.19
55,동작구,상도1동,11590530,0.594856,0.686916,0.472196,0.574032,1.16


In [23]:
# minMax스케일링
fin_features = ['자치구', '행정동', '행정동_코드', '1인가구_비율', '연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수']
df = df[fin_features]

scaler = MinMaxScaler()
df.iloc[:, 3:] = scaler.fit_transform(df.iloc[:, 3:])
df['평가점수'] = df['1인가구_비율']*0.65 + df['연령대_2030_거주인구수_합']*0.60 + df['주요경제활동인구_생활인구수']*0.46
df['평가점수'] = np.round(df['평가점수'], 2)
# df.head()
df.sort_values(by='평가점수', ascending = False).head()

,자치구,행정동,행정동_코드,1인가구_비율,연령대_2030_거주인구수_합,주요경제활동인구_생활인구수,수거함개수_당_거주인구수,평가점수
60,관악구,청룡동,11620595,0.988142,0.878827,0.468999,0.245140,1.39
51,영등포구,영등포동,11560535,1.000000,0.600803,0.745670,0.010561,1.35
41,강서구,화곡1동,11500540,0.581286,1.000000,0.573449,0.849013,1.24
76,송파구,문정2동,11710642,0.752118,0.452604,0.576903,0.018222,1.03
55,동작구,상도1동,11590530,0.594856,0.686916,0.472196,0.574032,1.02


In [9]:
df.to_csv('행정동_지표포함.csv', encoding = 'utf-8-sig', index = False)

In [20]:
len(df['행정동_코드'].value_counts())

85